In [45]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
%matplotlib inline
from skimage import io, transform
import glob
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler


In [40]:
class NotMnist(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.filelist = glob.glob(os.path.join(self.root_dir, '**', '*.png'))
        new_filelist = []
        for file in self.filelist:
            try:
                io.imread(file)
                new_filelist.append(file)
            except:
                pass
                
        self.filelist = new_filelist
        
    def __len__(self):
        return len(self.filelist)
        
    def __getitem__(self, idx):
        image = io.imread(self.filelist[idx])
        image = PIL.Image.fromarray(image)
        if self.transform:
            return self.transform(image)
        return image

In [57]:
## treat notMNIST_Larget as train set (which further would be splitted into training_set and validation_set), and treat notMNIST_small as test set
image_transforms=transforms.Compose([transforms.ToTensor()])
train_set=NotMnist('notMNIST_large',transform=image_transforms)
batch_size = 2

In [58]:
num_train = len(train_set)
indices = list(range(num_train))
split = 4

# Random, non-contiguous split
validation_idx = np.random.choice(indices, size=split, replace=False)
train_idx = list(set(indices) - set(validation_idx))

# Contiguous split
# train_idx, validation_idx = indices[split:], indices[:split]

## define our samplers -- we use a SubsetRandomSampler because it will return
## a random subset of the split defined by the given indices without replaf
train_sampler = SubsetRandomSampler(train_idx)
validation_sampler = SubsetRandomSampler(validation_idx)

train_loader = torch.utils.data.DataLoader(train_set, 
                batch_size=2, sampler=train_sampler)

validation_loader = torch.utils.data.DataLoader(train_set, 
                batch_size=2, sampler=validation_sampler)

In [59]:
#train_loader = torch.utils.data.DataLoader(NotMnist('notMNIST_large',
#                                                      transform=image_transforms), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(NotMnist('notMNIST_small',
                                                      transform=image_transforms), batch_size=batch_size, shuffle=True)


In [60]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [61]:
model = CNN()

optimizer = optim.SGD(model.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss()

for epoch in range(20):
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.shape[0]
        correct_cnt+= (pred_label == target).sum().item()
        ave_loss = ave_loss * 0.9 + loss.item() * 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print('==>>> epoch: {}, batch index: {}, train loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, ave_loss, correct_cnt*1.0/total_cnt))
    # testing
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.shape[0]
#         print(target.data)
        correct_cnt += (pred_label == target).sum().item()
        # smooth average
        ave_loss = ave_loss * 0.9 + loss.item() * 0.1
        
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_loader):
            print('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, ave_loss, correct_cnt * 1.0 / total_cnt))
    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct_cnt / total_cnt) * 100))


# Save the model and plot
torch.save(model.state_dict(),'./conv_net_model.ckpt')

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 1, 5, 5], but got input of size [1, 28, 28] instead

0